In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, os, django

sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from tmv_app.models import *

django.db.connection.close()

In [18]:
run_id = 2927 

dts = pd.DataFrame(DocTopic.objects.filter(run_id=run_id).values('topic_id','doc_id','score')).sort_values('doc_id').dropna()
dts.head()



,topic_id,doc_id,score
242776,213165,103.0,0.000179
287770,213173,103.0,0.231720
186031,213151,103.0,0.037914
61677,213139,103.0,0.079174
103157,213146,103.0,0.008028


In [58]:
dtm = dts.pivot(index="doc_id",columns="topic_id")
W = dtm.fillna(0).values
print(W.shape)
dtm

(15963, 70)


score                                                           \
topic_id     213122    213123    213124    213125    213126    213127 213128   
doc_id                                                                         
103.0           NaN       NaN       NaN  0.009277       NaN       NaN    NaN   
562.0      0.067808  0.009821  0.011236  0.015867       NaN       NaN    NaN   
1061.0          NaN  0.012908       NaN       NaN       NaN       NaN    NaN   
1285.0     0.002073  0.020111       NaN  0.010156       NaN  0.000143    NaN   
1672.0     0.032696       NaN       NaN  0.001745       NaN       NaN    NaN   
...             ...       ...       ...       ...       ...       ...    ...   
3760043.0  0.013610       NaN       NaN       NaN       NaN       NaN    NaN   
3760056.0       NaN  0.017209       NaN  0.001791       NaN       NaN    NaN   
3760070.0  0.004025  0.028156       NaN  0.001372  0.001285       NaN    NaN   
3760097.0       NaN       NaN       NaN       NaN  0.000199       NaN    NaN   
3760150.0       NaN       NaN       NaN       NaN       NaN       NaN    NaN   

                                         ...                                \
topic_id     213129    213130    213131  ...    213182    213183    213184   
doc_id                                   ...                                 
103.0           NaN       NaN       NaN  ...       NaN       NaN       NaN   
562.0           NaN       NaN  0.000024  ...  0.000568       NaN       NaN   
1061.0          NaN  0.019703       NaN  ...       NaN       NaN  0.006059   
1285.0          NaN  0.023758       NaN  ...       NaN       NaN       NaN   
1672.0          NaN       NaN  0.000621  ...  0.052082  0.000876       NaN   
...             ...       ...       ...  ...       ...       ...       ...   
3760043.0  0.005353       NaN  0.008294  ...       NaN       NaN       NaN   
3760056.0       NaN  0.049981       NaN  ...  0.000266       NaN       NaN   
3760070.0       NaN  0.002043       NaN  ...       NaN  0.193119       NaN   
3760097.0       NaN       NaN       NaN  ...  0.000748       NaN       NaN   
3760150.0       NaN  0.000418       NaN  ...       NaN       NaN       NaN   

                                                                              
topic_id     213185    213186    213187    213188 213189    213190    213191  
doc_id                                                                        
103.0           NaN       NaN  0.018811       NaN    NaN       NaN       NaN  
562.0           NaN       NaN       NaN  0.000556    NaN  0.008197       NaN  
1061.0          NaN       NaN       NaN       NaN    NaN       NaN  0.008945  
1285.0     0.043741       NaN       NaN  0.022516    NaN       NaN       NaN  
1672.0          NaN       NaN       NaN  0.010383    NaN       NaN       NaN  
...             ...       ...       ...       ...    ...       ...       ...  
3760043.0       NaN  0.024955       NaN       NaN    NaN       NaN  0.006169  
3760056.0  0.009084  0.000139       NaN  0.015485    NaN       NaN       NaN  
3760070.0       NaN       NaN       NaN       NaN    NaN  0.007105       NaN  
3760097.0  0.008054       NaN       NaN  0.000937    NaN       NaN       NaN  
3760150.0       NaN       NaN       NaN  0.006381    NaN       NaN       NaN  

[15963 rows x 70 columns]

In [6]:

tts = pd.DataFrame(TopicTerm.objects.filter(run_id=run_id).values('topic_id','term_id','score')).sort_values('topic_id')
tts.head()

,topic_id,term_id,score
0,213129,179,0.003257
1,213133,179,0.000373
2,213137,179,0.006755
3,213139,179,0.007502
4,213143,179,0.000911


In [102]:
vocab = pd.DataFrame(Term.objects.filter(id__in=tts['term_id'].unique()).values('title','id'))
vocab

,title,id
0,account,1
1,activ,2
2,addit,3
3,affect,4
4,also,5
...,...,...
6404,sandyrel,2396893
6405,particulatematterassoci,2420430
6406,particulatematterbound,2420431
6407,particulatematterparticulatematt,2420432


In [53]:
vocab[vocab['title']=="pollut"]

,title,id
3328,pollut,6920


In [31]:
ttm = tts.pivot(index="topic_id",columns="term_id")
H = ttm.fillna(0).values
H.shape

(70, 6409)

In [24]:
W.shape

(15963, 70)

In [55]:
X = np.random.random((5,H.shape[1]))
X = np.zeros((5,H.shape[1]))
X[:,3328] = 1
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [56]:
from sklearn.decomposition import NMF

clf = NMF(
    n_components=ttm.shape[0],
    init="custom",
    max_iter=5
)

clf.components_ = H
clf.n_components_ = H.shape[0]

clf.transform(
    X
)

array([[0.        , 0.17085547, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.00109417, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.00037292],
       [0.        , 0.17085547, 0.        , 0. 

In [ ]:
def proc_docs(docs, stoplist, fulltext=False, citations=False):

    docs = [x for x in docs if x.str.findall(r'(\w+)').str.len() > 10]
    if fulltext:
        abstracts = [x.fulltext.replace("Copyright (C)","") for x in docs]
    else:
        abstracts = [x.content.split("Copyright (C)")[0] for x in docs]
        abstracts = [re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",x)[0] for x in abstracts]
        abstracts = [x.split("Published by Elsevier")[0] for x in abstracts]
        abstracts = [x.split("Copyright. (C)")[0] for x in abstracts]
        abstracts = [re.split("\. \(C\) [1-2][0-9]{3} ",x)[0] for x in abstracts]
        abstracts = [re.split("\. \(C\) Copyright",x)[0] for x in abstracts]
    docsizes = [len(x) for x in abstracts]
    ids = [x.pk for x in docs]

    if citations:
        citations = []
        for x in docs:
            try:
                if x.wosarticle.tc is not None:
                    citations.append(x.wosarticle.tc)
                else:
                    citations.append(0)
            except:
                citations.append(0)
    return [abstracts, docsizes, ids, citations]

In [61]:
stat = RunStats.objects.get(run_id=run_id)
stat.__dict__

{'K': 70,
 '_state': <django.db.models.base.ModelState at 0x7f8b18f9a9b0>,
 'alpha': 0.1,
 'batch_count': 0,
 'beta': None,
 'citations': False,
 'coherence': 0.2056005073008762,
 'db': True,
 'db_time': 48.6634955406189,
 'doc_topic_scaled_score': False,
 'docs_seen': 16078,
 'dt_threshold': 0.01,
 'dt_threshold_scaled': 0.01,
 'dyn_win_threshold': 0.1,
 'empty_topics': 0,
 'error': 109.8139889672442,
 'errortype': 'Frobenius',
 'exclusivity': None,
 'extra_stopwords': None,
 'fancy_tokenization': False,
 'fulltext': False,
 'iterations': None,
 'language': None,
 'last_update': datetime.datetime(2020, 7, 8, 19, 2, 11, 580786, tzinfo=<UTC>),
 'lda_learning_method': 'ba',
 'lda_library': 'sk',
 'limit': 0,
 'max_df': 0.95,
 'max_features': 0,
 'max_iter': 200,
 'max_topics': None,
 'method': 'NM',
 'min_freq': 10,
 'ngram': 1,
 'nmf_time': 30.959132194519043,
 'notes': None,
 'parent_run_id': 2927,
 'process_id': 119288,
 'psearch_id': None,
 'query_id': 8530,
 'rng_seed': None,
 'run_

In [ ]:
abstracts, docsizes, ids, citations = proc_docs(docs, stoplist, stat.fulltext, stat.citations)

In [59]:
seen_df = pd.read_csv('data/0a_labelled_documents.csv')
seen_df.head()

,id,content,title,PY,wosarticle__de,wosarticle__wc,relevant,seen,1 - Adaptation,1 - Impact,1 - Mitigation
0,282602,Background: Among the many challenges faced by...,Health coping strategies of the people vulnera...,2013.0,Health coping strategies; Choice of care; Unqu...,"['Public, Environmental & Occupational Health']",1.0,1.0,1.0,0.0,0.0
1,773200,The projected rise in food-related greenhouse ...,Mitigation potential and global health impacts...,2017.0,0,['Environmental Sciences; Environmental Studie...,1.0,1.0,0.0,0.0,1.0
2,3446429,Background: Dengue fever control in the tropic...,Spatial and temporal variation of dengue incid...,2019.0,Dengue; Bali; Spatial analysis; Conditional au...,"['Public, Environmental & Occupational Health;...",1.0,1.0,0.0,1.0,0.0
3,1672609,Vector-borne infectious diseases continue to b...,Gene drives as a response to infection and res...,2019.0,CRISPR; gene drive; gene editing; vector-borne...,['Infectious Diseases; Pharmacology & Pharmacy'],0.0,1.0,0.0,0.0,0.0
4,3393270,A chronic symptomatic acromioclavicular joint ...,Arthroscopically assisted stabilization of chr...,2013.0,Chronic acromioclavicular joint dislocation; A...,['Orthopedics; Surgery'],0.0,1.0,0.0,0.0,0.0


In [ ]:
def proc_docs(docs, stoplist, fulltext=False, citations=False):

    docs = [x for x in docs if x.str.findall(r'(\w+)').str.len() > 10]
    if fulltext:
        abstracts = [x.fulltext.replace("Copyright (C)","") for x in docs]
    else:
        abstracts = [x.content.split("Copyright (C)")[0] for x in docs]
        abstracts = [re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",x)[0] for x in abstracts]
        abstracts = [x.split("Published by Elsevier")[0] for x in abstracts]
        abstracts = [x.split("Copyright. (C)")[0] for x in abstracts]
        abstracts = [re.split("\. \(C\) [1-2][0-9]{3} ",x)[0] for x in abstracts]
        abstracts = [re.split("\. \(C\) Copyright",x)[0] for x in abstracts]
    docsizes = [len(x) for x in abstracts]
    ids = [x.pk for x in docs]

    if citations:
        citations = []
        for x in docs:
            try:
                if x.wosarticle.tc is not None:
                    citations.append(x.wosarticle.tc)
                else:
                    citations.append(0)
            except:
                citations.append(0)
    return [abstracts, docsizes, ids, citations]

In [88]:
seen_df['n_words'] = seen_df['content'].str.findall(r'(\w+)').str.len()
docs = seen_df[seen_df['n_words']>10]
docs['content'] = [x.split("Copyright (C)")[0] for x in docs['content']]
docs['content'] = [re.split("\([C-c]\) [1-2][0-9]{3} Elsevier", x)[0] for x in docs['content']]
docs['content'] = [x.split("Published by Elsevier")[0] for x in docs['content']]
docs['content'] = [x.split("Copyright. (C)")[0] for x in docs['content']]
docs['content'] = [re.split("\. \(C\) [1-2][0-9]{3} ", x)[0] for x in docs['content']]
docs['content'] = [re.split("\. \(C\) Copyright", x)[0] for x in docs['content']]

ids = docs['id']
abstracts = docs['content']

In [103]:
from nltk.stem import SnowballStemmer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string

def tokenize(text):
    transtable = {ord(c): None for c in string.punctuation + string.digits}
    tokens = nltk.word_tokenize(text.translate(transtable))
    tokens = [i for i in tokens if len(i) > 2 and len(i) < 100]
    return tokens

class snowball_stemmer(object):
    def __init__(self):
        self.stemmer = SnowballStemmer("english")
    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in tokenize(doc)]

tokenizer = snowball_stemmer()
stoplist = set(nltk.corpus.stopwords.words("english"))

vectorizer = TfidfVectorizer(
    max_df=stat.max_df,
    min_df=stat.min_freq,
    ngram_range=(stat.ngram,stat.ngram),
    tokenizer=tokenizer,
    stop_words=stoplist
)
tfidf = vectorizer.fit_transform(abstracts).todense()
vocab_list = vectorizer.get_feature_names()

/home/max/software/django-tmv/tmv/venv/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'arent', 'becaus', 'befor', 'couldnt', 'didnt', 'doe', 'doesnt', 'dont', 'dure', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'onc', 'onli', 'ourselv', 'shant', 'shes', 'shouldnt', 'shouldv', 'thatll', 'themselv', 'veri', 'wasnt', 'werent', 'whi', 'wont', 'wouldnt', 'youd', 'youll', 'yourselv', 'youv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [121]:
X = np.matrix(np.zeros((len(docs),H.shape[1])))
X.shape

(3240, 6409)

In [109]:
vocab_translate = {row['title']: i for i, row in vocab.iterrows()}
for i, word in enumerate(vocab_list):
    try:
        term_idx = vocab_translate[word]
    except KeyError as e:
        error = f"couldn't find {e}"
    X[:,term_idx] = tfidf[:,i]

In [ ]:
W = clf.transform(
    X
)